In [ ]:
import sqlite3 as sql
import random
import logging

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

formatter = logging.Formatter('%(asctime)s:%(name)s:%(message)s')

file_handler=logging.FileHandler('chatBank.log')
file_handler.setFormatter(formatter)

logger.addHandler(file_handler)


class Employee:

    def __init__(self,dbpath,employee_id=0,employee_name='',salary=0): 
        """Default constructor of Employee class, assigns values to all the aruguments. 
           This creates SQLite Table employee first time of its execution.
           Also creates a SQLite DB connection and opens a cursor and before leaving the method closes them.
           
           Args:
               self:               Employee Class instance
               employee_id(INT):   Employee Id
               employee_name(STR): Employee Name
               salary(FLOAT):      Employee's salary 
               
        """
        self.dbpath=dbpath
        connection = sql.connect(self.dbpath)
        Ecursor = connection.cursor()  
        self.employee_id = employee_id
        self.employee_name = employee_name
        self.salary = salary
        Ecursor.execute("CREATE TABLE if not exists employee(empid INT PRIMARY KEY,emp_name TEXT,salary DECIMAL(10,5))")
        connection.commit()
        Ecursor.close()
        connection.close()
    
    def add_employee(self):
        """this is a method to add employee details. Employee id is an integer value, start from 1 and is incremented
        for every employee id. Bank Manager with employee Ids 1 thru 5 can add employee details. 
        
            this method open a SQLite DB connection and a cursor for all DB queries and closes them out before leaving
            the method.
        Args:
            self:                  Employee Class instance
        """
        connection = sql.connect(self.dbpath)
        Ecursor = connection.cursor()  
        try:
            self.employee_id = int(input("Enter your Bank Employee Id: "))
        except ValueError:
            print("Invalid Employee Id entered")
            logger.exception("Invalid Employee Id")
            Ecursor.close()
            connection.close()
            return ('T')
        Ecursor.execute("SELECT empid FROM employee WHERE empid = ?",(self.employee_id,))
        records = Ecursor.fetchall()
        if len(records) > 0:
            pass
        else:
            print("Employee does not exist")
            logger.debug("Employee doesnt exist")
            Ecursor.close()
            connection.close()
            return ('T')

        chk_empid = records[0][0]
        if (chk_empid >= 1 and chk_empid <=5):
            pass
        elif chk_empid > 5:
            print("Employee data can only be accessed by managers")
            print_stars()
            Ecursor.close()
            connection.close()
            return ('T')
        elif chk_empid == None:
            print("Invalid Employee Id")
            print_stars()
            Ecursor.close()
            connection.close()
            return ('T')
            
        self.employee_name = input("Enter the full name of new Employee: ")
        if self.employee_name == None:
            logger.debug("Enter valid name")
        try:
            self.salary = float(input("Enter employee's Salary: "))
        except ValueError:
            logger.exception("Enter valid Salary")
            Ecursor.close()
            connection.close()
            return ('T')
        Ecursor.execute("SELECT max(empid) FROM employee")
        records = Ecursor.fetchall()
        max_empid = records[0][0]
        if max_empid == None:
            max_empid = 0
        self.employee_id = int(max_empid) + 1
        Ecursor.execute("INSERT INTO employee VALUES (?,?,?)",(self.employee_id,self.employee_name,self.salary,))
        connection.commit()
        print("\nEmployee is now added")
        
        print_stars()
        for row in Ecursor.execute("SELECT * FROM employee WHERE emp_name = ?",(self.employee_name,)):
            print(row)
        connection.commit()
        Ecursor.close()
        connection.close()
        
        return ('T') 
    
    def upd_salary(self):
        """this method updates Employee's Salary based on the Employee Id entered. Only Bank Managers 
           with Employee Ids 1 thru 5 can update Salary.
           
           this method open a SQLite DB connection and a cursor for all DB queries and closes them out before leaving
            the method.
           Args:
                self:                  Employee Class instance
        """
        connection = sql.connect(self.dbpath)
        Ecursor = connection.cursor()  
        try:
            self.employee_id = int(input("Enter your Bank Employee Id: "))
        except ValueError:
            print("Invalid Employee Id entered")
            logger.exception("Invalid Employee Id")
            Ecursor.close()
            connection.close()
            return ('T')
        Ecursor.execute("SELECT empid FROM employee WHERE empid = ?",(self.employee_id,))
        records = Ecursor.fetchall()
        chk_empid = records[0][0]
        if (chk_empid >= 1 and chk_empid <=5):
            pass
        elif chk_empid > 5:
            print("Employee data can only be accessed by managers")
            print_stars()
            Ecursor.close()
            connection.close()
            return ('T')
        elif chk_empid == None:
            print("Invalid Employee Id")
            print_stars()
            Ecursor.close()
            connection.close()
            return ('T')
        try:
            self.employee_id = int(input("Enter Employee Id: "))
            self.salary = float(input("Enter employee's Salary to be updated: "))
        except ValueError:
            logger.exception("Valid employee values are not entered")
            print_stars()
            connection.commit()
            Ecursor.close()
            connection.close()
            return ('T')
      
        Ecursor.execute("SELECT salary FROM employee WHERE empid = ?",(self.employee_id,))
        records = Ecursor.fetchall()
        emp_sal = records[0][0]
        if emp_sal == None:
            print("Invalid Employee Id")
            connection.commit()
            Ecursor.close()
            connection.close()
            return ('T')
 
    
        Ecursor.execute("UPDATE employee SET salary = ? WHERE empid = ? ",(self.salary,self.employee_id,))
        connection.commit()
        print("\nEmployee's salary is updated")
        
        print_stars()
        for row in Ecursor.execute("SELECT * FROM employee WHERE empid = ?",(self.employee_id,)):
            print(row)
        connection.commit()
        Ecursor.close()
        connection.close()

        return ('T') 

    def get_employee(self):
        """this method fetches Employee details based on the Employee Id entered. Only Bank Managers 
           with Employee Ids 1 thru 5 can update Salary.
           
           this method opens a SQLite DB connection and a cursor for all DB queries and closes them out before leaving
            the method.
           Args:
                self:                  Employee Class instance
        """
        connection = sql.connect(self.dbpath)
        Ecursor = connection.cursor()  

        self.employee_name = input("Enter Employee name: ")
        Ecursor.execute("SELECT * FROM employee WHERE emp_name = ?",(self.employee_name,))
        records = Ecursor.fetchall()
        if len(records) == None:
            print("Employee with name "+self.employee_name+"doesnt exist")
            logger.info("Employee with name "+self.employee_name+"doesnt exist")
            print_stars()
 
            connection.commit()
            Ecursor.close()
            connection.close()
            return ('T')
        
        for row in records:
            print(row)

        connection.commit()
        Ecursor.close()
        connection.close()

        return ('T') 
    
class Services:
        
    def __init__(self,dbpath,account_id=0,account_type='',balance=0,customer_name='',customer_addr='',):   
       
        """Default constructor of Services class, initializes all the aruguments. 
           
           Args:
               self:               Services Class instance
               account_id(INT):    Account Id
               account_type(STR):  Account Type (Savings or Checking)
               employee_name(STR): Customer Name
               salary(STR):      Cutomer address 
               
        """
        self.dbpath = dbpath
        self.account_id = account_id
        self.account_type = account_type
        self.balance = balance
        self.customer_name = customer_name
        self.customer_addr = customer_addr
    
    def add_deposit(self):
        
        """this method adds funds to the Account Id entered.
           
           Args:
               self:               Services Class instance
        """
        connection = sql.connect(self.dbpath)
        Scursor = connection.cursor()  
        try:
            acct_id = int(input("Enter 10 digit Account Id : "))
        except ValueError:
            logger.debug("Enter a valid account id")
            Scursor.close()
            connection.close()
            return ('T')
        Scursor.execute("SELECT balance FROM accounts WHERE acctid = ?",(acct_id,))
        records = Scursor.fetchall()
        if len(records) == 1:
            bal = records[0][0]
        elif len(records) == 0:
            print("Account Id "+str(acct_id)+" does not exist!\n")
            return ('T')
        try:
            amount = float(input("Enter amount to be Deposited: "))
        except ValueError:
            logger.debug("a valid amount should be entered")
            Scursor.close()
            connection.close()
            return('T')
        bal += amount
        print_stars()
        
        Scursor.execute("UPDATE accounts SET balance = ? WHERE acctid = ?",(bal,acct_id,))
        connection.commit()
        for row in Scursor.execute("SELECT * FROM accounts WHERE acctid = ?",(acct_id,)):
            print(row)
        print("\nAccount "+str(acct_id)+" has been Deposited with $"+str(amount))
        print("Current available balance is: $"+str(bal)+"\n")
        
        print_stars()
        Scursor.close()
        connection.close()
        return ('T')
    
    def withdraw(self):
        """this method withdraws funds from the Account Id entered.
           
           Args:
               self:               Services Class instance
        """
        connection = sql.connect(self.dbpath)
        Scursor = connection.cursor()  
        try:
            acct_id = int(input("Enter 10 digit Account Id to Withdraw from: "))
        except ValueError:
            logger.debug("Valid Account Id must be entered")
            Scursor.close()
            connection.close()
            return 'T'
        Scursor.execute("SELECT balance FROM accounts WHERE acctid = ?",(acct_id,))
        records = Scursor.fetchall()
        if len(records) == 1:
            bal = records[0][0]
        elif len(records) == 0:
            print("Account Id "+str(acct_id)+" does not exist!")
            return ('T')
        try:
            amount = float(input("Enter amount to be Withdrawn: "))
        except ValueError:
            logger.debug("Enter a valid amount value")
            Scursor.close()
            connection.close()
            return ('T')
        if bal >= amount:
            bal -= amount
        else:
            print("Insufficient amounts in Account :"+str(acct_id)+"\n")
            print_stars()
            return ('T')
        Scursor.execute("UPDATE accounts SET balance = ? WHERE acctid = ?",(bal,acct_id,))
        connection.commit()
        print("\nAmount successfully withdrawn")
        for row in Scursor.execute("SELECT * FROM accounts WHERE acctid = ?",(acct_id,)):
            print(row)
        print("Current available balance is: $"+str(bal)+"\n")
        print_stars()

        Scursor.close()
        connection.close()
        return ('T')

    def chk_balance(self):
        connection = sql.connect(self.dbpath)
        Scursor = connection.cursor()
        try:
            acct_id = int(input("Enter 10 digit Account Id to check balance: "))
        except ValueError:
            logger.debug("Valid account id should be entered")
            Scursor.close()
            connection.close()
            return ('T')
        Scursor.execute("SELECT balance FROM accounts WHERE acctid = ?",(acct_id,))
        records = Scursor.fetchall()
        if len(records) == 1:
            bal = records[0][0]
        elif len(records) == 0:
            print("Account Id "+str(acct_id)+" does not exist!")
            print_stars()
            return ('T')
        for row in Scursor.execute("SELECT * FROM accounts WHERE acctid = ?",(acct_id,)):
            print(row)
        print("Current available balance is: $"+str(bal)+"\n")
        print_stars()

        Scursor.close()
        connection.close()
        return ('T')

    
class Accounts:

    def __init__(self,dbpath,account_id=0,account_type='',balance=0,customer_name='',customer_addr='',): 

        """Default constructor of Accounts class, initalized all the aruguments. 
           This creates SQLite Table accounts first time of its execution.
           Also creates a SQLite DB connection and opens a cursor and before leaving the method closes them.
           
           Args:
               self:               Employee Class instance
               employee_id(INT):   Employee Id
               employee_name(STR): Employee Name
               salary(FLOAT):      Employee's salary 
               
        """
        self.dbpath=dbpath
        connection = sql.connect(self.dbpath)
        cursor = connection.cursor()  
        
        self.account_id = account_id
        self.account_type = account_type
        self.balance = balance
        self.customer_name = customer_name
        self.customer_addr = customer_addr
        cursor.execute("CREATE TABLE if not exists accounts(acctid TEXT PRIMARY KEY,acct_type TEXT,balance DECIMAL(10,5),cust_name TEXT,cust_addr TEXT)")
        connection.commit()
        cursor.close()
        connection.close()
    
    def add_savings(self):
        connection = sql.connect(self.dbpath)
        cursor = connection.cursor()  
  
        customer_name = input("Enter the full name of new Account holder: ")
        customer_addr = input("Enter full address of Account holder: ")
        atype = input("What type of an Account do you want, ''1.Savings 2.Checking 3.Both(Savings&Checking)''")
        if atype == '1':
            self.account_type = "Savings"
            a1="649"
            a2=str(random.randrange(100000,999999))
            a3="1"
            self.account_id=a1+a2+a3
            print(self.account_id)
        elif atype == '2':
            self.account_type = "Checking"
            a1="649"
            a2=str(random.randrange(100000,999999))
            a3="2"
            self.account_id=a1+a2+a3
        elif atype == '3':
            self.account_type = "Savings"
            a1="649"
            a2=str(random.randrange(100000,999999))
            a3="1"
            self.account_id=a1+a2+a3
            cursor.execute("INSERT INTO accounts VALUES (?,?,?,?,?)",(self.account_id,self.account_type,self.balance,customer_name,customer_addr,))
            connection.commit()
            self.account_type = "Checking"
            a3="2"
            self.account_id=a1+a2+a3
        else:
            print("Invalid option entered")
            print_stars()
            return ('T')
        
        cursor.execute("INSERT INTO accounts VALUES (?,?,?,?,?)",(self.account_id,self.account_type,self.balance,customer_name,customer_addr,))
        connection.commit()
        print("\nAccount has been successfully added")
        
        for row in cursor.execute("SELECT * FROM accounts WHERE cust_name = ?",(customer_name,)):
            print(row)
        print_stars()
        return ('T')    

def exit_menu():
    #a1.add_savings.cursor.close()
    #a1.add_savings.connection.close()
    print("                  Have a Good Day!")
    return('F')
    
def invalid_opt():
    print("Invalid Option")
    #a1.add_savings.cursor.close()
    #a1.add_savings.connection.close()

    return ('T')

def print_stars():
    print("\n")
    print("*"*20)
    print("\n")

print("                                  Welcome to chatBank! ")
dbpath = input("Please enter a path of your local SQlite Database: ")

e1=Employee(dbpath)
a1=Accounts(dbpath)
s1=Services(dbpath)

print("                               How can we help you today?")
print_stars()
menu = {"1":("Open a new Account",a1.add_savings),
        "2":("Deposit",s1.add_deposit),
        "3":("Withdraw",s1.withdraw),
        "4":("Check Balance",s1.chk_balance),
        "5":("Add Employee Data (Bank Manager access)",e1.add_employee),
        "6":("Update Employee's Salary (Bank Manager access)",e1.upd_salary),
        "7":("Get Employee Data (Bank Manager access)",e1.get_employee),
        "8":("Exit",exit_menu)}
wl = 'T'
while (wl=='T'):
    for key in sorted(menu.keys()):
        print(key+":"+menu[key][0])
    print_stars()
    ans = input("Make a choice - ")
    wl=menu.get(ans,[None,invalid_opt])[1]()
    print_stars()
    



                                  Welcome to chatBank! 


README
"""Project Title: chatBank, a simple Banking System using Python-SQlite.
 
Description: Here is a simple Banking System coded in Python, that allows users to create Savings and    Checking accounts or eiher one of them. This is an interactive system that asks for inputs from users, processes data, stores the processed data in SQLite tables and logs exceptions into a file.
    Having developed end to end projects in Legacy Mainframe based applications, this is my first ever desktop app using modern concepts like Object Oriented programming, and tools like SQLite, DBeaver, GitHub, Jupyter Notebook.
 
Challenges and scope of improvement: Have tried MYSql but have encountered system issues and had to switch to SQLite. In future I want to code using Prostgres or MySQL databases.

How to Setup and run:
        1. SQLite database is needed. Create a folder for Banking tables to be created.
        2. SQLite database path is the first input that system takes in.
        3. Errors and exceptions will be logged into file chatBank.log

Features of Banking System: 
       this is a simple banking system that creates accounts, lets users deposit and withdraw amount. Also maintains Employees data which can be accessed by Bank Managers only and their Employee Ids are 1,2,3,4 or 5.
"""

